<span STYLE="font-size:150%"> 
    Segment polygonal regions from microCT scans
</span>

Docker image: gnasello/slicer-env:2023-10-10 \
Latest update: 11 October 2023

- load image stack in Slicer
- segment mineralized tissue
- compute segmented statistics (volumes)

# Load libraries

In [ ]:
import pyslicer as ps
import slicer
from pathlib import Path
import pandas as pd
import trimesh
import vtk
import numpy as np

# Volume input

## Load `.nrrd` file into 3D Slicer

Write the path of the `.nrrd` file and load it to Slicer

In [ ]:
# this cell is tagged 'parameters'
volume_file = 'microCT_volume_preview/microCT_volume_preview.nrrd'
output_polygon_dir_path = 'segmented_volumes/polygonDefect'
output_cylinder_dir_path = 'segmented_volumes/originalDefect'
file_segmentation = 'segmented_volumes/Skull.seg.nrrd'
directory_notebook = Path().parent.absolute()
sample_name = directory_notebook.stem
camera_view_file = 'segmented_volumes/camera_view.csv'

In [ ]:
masterVolumeNode = slicer.util.loadNodeFromFile(volume_file)

Print spacing

In [ ]:
## mm
masterVolumeNode.GetSpacing()

Make ```segmented_volumes``` folder

In [ ]:
output_originalDefect_path = Path(output_cylinder_dir_path)

output_originalDefect_path.mkdir(parents=True, exist_ok=True)

In [ ]:
output_polygonDefect_path = Path(output_polygon_dir_path)

Get norebook directory name

In [ ]:
sample_name

# Create segmentationNode

## Load Bone Segment as segmentationNode

In [ ]:
segmentationNode = slicer.util.loadSegmentation(str(Path(file_segmentation).resolve()), properties={"name": "Segmentation"})

Rename segment of the segmentation node 

In [ ]:
segmentation = segmentationNode.GetSegmentation()
segment = segmentation.GetNthSegment(0)
segment.SetName("Skull")

Create temporary segment editor to get access to effects

In [ ]:
segmentEditorWidget, segmentEditorNode = ps.segmentation.segmentEditorWidget(segmentationNode = segmentationNode, 
                                                                             masterVolumeNode = masterVolumeNode)

Get closed surface representation of the segment, from [slicer scripting repository](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-nodes-warped-by-transform-sequence)

In [ ]:
segmentationNode.CreateClosedSurfaceRepresentation()

In [ ]:
segments_color = {
    "Skull": (0.9450980392156862, 0.8392156862745098, 0.5686274509803921), # "Bone" color in Slicer
    }

segments_color

In [ ]:
ps.segmentation.set_segments_color(segments_color, segmentationNode)

# Visualize segmentations in 3D view

In [ ]:
# Switch to "One 3D view" layout
layoutManager = slicer.app.layoutManager()
layoutManager.setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutOneUp3DView)

# If you want to return later to the default layout (4-up), you can use:
# layoutManager.setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutFourUpView)

In [ ]:
ps.view.default_dark_3D_view()

## Get camera position in 3D view

Load `camera_view.csv` file

In [ ]:
df_camera = pd.read_csv(camera_view_file)
df_camera

In [ ]:
ps.view.set_camera_3Dview(position=df_camera['position'],
                          viewAngle=df_camera['viewAngle'][0],
                          viewUp=df_camera['viewUp'],
                          parallelScale=df_camera['parallelScale'][0])

# Define polygon defect 

## Set-up the markups module

In [ ]:
# Show Markups module
slicer.util.selectModule('Markups')

# Create new fiducial list named "F"
markupsLogic = slicer.modules.markups.logic()
fidNodeName = markupsLogic.AddNewFiducialNode()   # returns string name
fidList = slicer.util.getNode(fidNodeName)        # fetch node object
fidList.SetName("F")

# Show markups toolbar
slicer.util.mainWindow().findChild("qMRMLMarkupsToolBar").setVisible(True)

In [ ]:
# Enable "Place multiple control points"
selectionNode = slicer.app.applicationLogic().GetSelectionNode()
selectionNode.SetReferenceActivePlaceNodeClassName("vtkMRMLMarkupsFiducialNode")
slicer.app.applicationLogic().GetInteractionNode().SetPlaceModePersistence(1)  # allow multiple points
#slicer.app.applicationLogic().GetInteractionNode().SwitchToSinglePlaceMode()   # switch to placing

In [ ]:
# Switch to "One 3D view" layout
layoutManager = slicer.app.layoutManager()
layoutManager.setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutOneUp3DView)

# If you want to return later to the default layout (4-up), you can use:
# layoutManager.setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutFourUpView)

## Draw points in Slicer UI

Before running the code below, go to the Slicer UI and define a **Point List** from the ```markups``` module. To select more points, click on `Toggle Markups Toolbar` and select the option `Place multiple control points`.

Set the name of the Point List node to ```F```. Any name is actually fine, but you should change the line below accoridngly.

**IMPORTANT** - Points in the Point List node ```F``` must be sorted to draw the polygon. You can move points in the list from the GUI (click `Advanced` in the Control Points List.

In [ ]:
nodename = 'F'
pointsNode = slicer.util.getNode(nodename)

In [ ]:
f_output = 'F.mrk.json'

slicer.util.exportNode(pointsNode, output_originalDefect_path / f_output)

## Fit plane to PointList

In [ ]:
planeNode, plane_dict = ps.markup.fitPlane_from_pointMarkup(pointNodename = 'F', namePlane = 'FittingPlane')
plane_dict

In [ ]:
color = (68/255, 172/255, 100/255)
opacity = 0.4

modelDisplayNode = planeNode.GetDisplayNode()
modelDisplayNode.SetColor(color[0], color[1], color[2])
modelDisplayNode.SetOpacity(opacity)

# Create a cylinder covering the original defect area

In [ ]:
projected_points = ps.markup.project_markupPoints_to_plane(pointNodename='F',
                                                           plane_normal=plane_dict['normal'],
                                                           plane_origin=plane_dict['origin']
                                                          )

In [ ]:
# Compute distances from the origin in 2D
radii = np.linalg.norm(projected_points, axis=1)

# Cylinder radius = max distance
original_radius = radii.max()

print("radii:", radii)
print(f"Original Defect Radius = {original_radius:.3f}")

In [ ]:
defect_height = 0.8 #mm 

# Create a model in 3D Slicer with the bounding cylinder cylinder
original_defect_cylinder = ps.model.create_hollow_cylinder(height=defect_height, 
                                                           radius_inner=0, radius_outer=original_radius, space =5, 
                                                           center=plane_dict['origin'],
                                                           direction=plane_dict['normal'],
                                                           transform=False,
                                                           nameModel='OriginalDefect', 
                                                           color=(216/255, 101/255, 79/255), 
                                                           opacity=0.5)

# Export cylinder and properties

In [ ]:
sample_name

Defect cylinder diameter

In [ ]:
# Scalar values
data = {'sample':sample_name,
        'original_defect_radius':original_radius}

df_polygon = pd.DataFrame(data, index=[0])

df_polygon.to_csv(output_originalDefect_path / (sample_name + '_original_defect.csv'), index = False)

df_polygon

Original Defect model

In [ ]:
model = original_defect_cylinder

filename_output = model.GetName() + '.vtk'

slicer.util.exportNode(model, output_originalDefect_path / filename_output)

Get closed surface representation of the segment, from [slicer scripting repository](https://slicer.readthedocs.io/en/latest/developer_guide/script_repository.html#export-nodes-warped-by-transform-sequence)

In [ ]:
segmentationNode.CreateClosedSurfaceRepresentation()

In [ ]:
slicer.modules.segmentations.logic().ExportSegmentsClosedSurfaceRepresentationToFiles(str(output_polygonDefect_path), segmentationNode)